In [ ]:
%pylab inline

In [ ]:
import re
import torch

import collections

import numpy as np
import scipy.io
import scipy.ndimage

import PIL

import logging
logging.getLogger("PIL").setLevel(logging.INFO)

import common.plotting
import torch 
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import tensorflow

import os
import ot
import itertools
import datetime
import sys
from random import randint
from pyriemann.utils.distance import *

os.environ['TORCH_MODEL_ZOO'] =  os.environ['PYTORCH_DATA_PATH']

In [ ]:
def load_torch_config(filename):
    return torch.load('%s/%s' % (DST_CONFIG_DIR, filename))

def save_torch_config(data, filename):
    if not os.path.exists(DST_CONFIG_DIR):
        os.mkdir(DST_CONFIG_DIR)
    return torch.save(data, '%s/%s' % (DST_CONFIG_DIR, filename))

def load_torch_big_data(filename, samples):
    return torch.load('%s/%d/%s' % (DST_BIG_DATA_DIR, samples, filename))

def save_torch_big_data(data, filename, samples):
    if not os.path.exists(DST_BIG_DATA_DIR):
        os.mkdir(DST_BIG_DATA_DIR)
    if not os.path.exists('%s/%d' % (DST_BIG_DATA_DIR, samples)):
        os.mkdir('%s/%d' % (DST_BIG_DATA_DIR, samples))
    return torch.save(data, '%s/%d/%s' % (DST_BIG_DATA_DIR, samples, filename))

def to_np(x):
    if isinstance(x, Variable):
        x = x.data
    return x.cpu().numpy()

def to_variable(x, **kwargs):
    x = torch.from_numpy(x)
    if CUDA:
        x = x.cuda()
    return Variable(x, **kwargs)

def log(text):
    if not os.path.exists(DST_CONFIG_DIR):
        os.mkdir(DST_CONFIG_DIR)
    text = '%s | %s' % (datetime.datetime.now(), text)
    with open('%s/%s' % (DST_CONFIG_DIR, LOG_FILE), 'a') as file:
        file.write(text + '\n')
        file.flush()
    print(text)
    sys.stdout.flush()

class Reshape(nn.Module):
    def __init__(self, *args):
        super(Reshape, self).__init__()
        self.shape = args

    def forward(self, x):
        return x.view((x.size(0), ) + self.shape)

In [ ]:
import ot

def ground_matrix(n):
    x = []
    for i in range(n):
        for j in range(n):
            x.append([i, j])
    x = np.array(x)
    M = ot.dist(x, x, 'sqeuclidean')
    return M

def sqeuclidean_wasserstein_distance(x, y):
    x = to_np(x)
    y = to_np(y)
    M = ground_matrix(x.shape[1])
    x = x.reshape(x.shape[0], -1)
    y = y.reshape(y.shape[0], -1)
    return torch.Tensor([ot.emd2(x[i], y[i], M) for i in range(0, x.shape[0])])

In [ ]:
def load_mnist(samples):
    log('start reading mnist data')
    
    try:
        log('read images, labels, distances from files')
        images = load_torch_big_data('images.pt', samples)
        labels = load_torch_big_data('labels.pt', samples)
        distances = load_torch_big_data('distances.pt', samples)
    except Exception as e:
        log('error, calucalting new data')
        data_path = os.environ.get('PYTORCH_DATA_PATH', '../data')

        dataset = torchvision.datasets.MNIST(data_path, train=True, download=True)
        imagesList = dataset.train_data
        imagesList = imagesList.squeeze(1).double()
        imagesList = imagesList.div(imagesList.sum(1).sum(1).unsqueeze(1).unsqueeze(1))
        labelsList = dataset.train_labels

        indexes = set()
        while len(indexes) < samples:
            indexes.add((randint(0, imagesList.size(0)-1), randint(0, imagesList.size(0)-1)))
        indexes = [j for i in list(indexes) for j in i]

        images = torch.index_select(imagesList, 0, torch.LongTensor(indexes))
        labels = torch.index_select(labelsList, 0, torch.LongTensor(indexes))
        images = images.view(-1, 2, images.size(1), images.size(2))
        labels = labels.view(-1, 2)
        distances = sqeuclidean_wasserstein_distance(images[:, 0], images[:, 1])

        images = images.float()
        distances = distances.float()

        save_torch_big_data(images, 'images.pt', samples)
        save_torch_big_data(labels, 'labels.pt', samples)
        save_torch_big_data(distances, 'distances.pt', samples)

    log('%d samples count' % images.size(0))
    log('distances sum: %.2f' % distances.sum())
    log('distances min: %.2f' % distances.min())
    log('distances max: %.2f' % distances.max())

    n_train = int(images.size(0) * 0.7)
    n_valid = int(images.size(0) * 0.9)
    train_images = images[:n_train]
    train_labels = labels[:n_train]
    train_distances = distances[:n_train]

    valid_images = images[n_train:n_valid]
    valid_labels = labels[n_train:n_valid]
    valid_distances = distances[n_train:n_valid]

    test_images = images[n_valid:]
    test_labels = labels[n_valid:]
    test_distances = distances[n_valid:]

    log('finish loading')
    return (train_images, train_labels, train_distances, valid_images, valid_labels, valid_distances, test_images, test_labels, test_distances)

In [ ]:
def randIndex(classes):
    c = randint(0, len(classes) - 1)
    s = sum(x.size(0) for x in classes[:c])
    i = randint(0, classes[c].size(0) - 1)
    return s + i
    
def load_doodle(samples):
    log('start reading doodle data')
    
    try:
        log('read images, labels, distances from files')
        images = load_torch_big_data('images.pt', samples)
        labels = load_torch_big_data('labels.pt', samples)
        distances = load_torch_big_data('distances.pt', samples)
    except Exception as e:
        log('error, calucalting new data')
        data_path = os.environ.get('PYTORCH_DATA_PATH', '../data')

        catsImages = torch.from_numpy(np.load('/home/i233123/nn/nn_assignments/project/doodle/cat.npy')).view(-1, 28, 28)
        crabsImages = torch.from_numpy(np.load('/home/i233123/nn/nn_assignments/project/doodle/crab.npy')).view(-1, 28, 28)
        facesImages = torch.from_numpy(np.load('/home/i233123/nn/nn_assignments/project/doodle/face.npy')).view(-1, 28, 28)
        mnistImages = torchvision.datasets.MNIST(data_path, train=True, download=True).train_data

        catsImages = catsImages.double()
        crabsImages = crabsImages.double()
        facesImages = facesImages.double()
        mnistImages = mnistImages.squeeze(1).double()
        
        catsImages = catsImages.div(catsImages.sum(1).sum(1).unsqueeze(1).unsqueeze(1))
        crabsImages = crabsImages.div(crabsImages.sum(1).sum(1).unsqueeze(1).unsqueeze(1))
        facesImages = facesImages.div(facesImages.sum(1).sum(1).unsqueeze(1).unsqueeze(1))
        mnistImages = mnistImages.div(mnistImages.sum(1).sum(1).unsqueeze(1).unsqueeze(1))               
        
        imagesList = torch.cat((mnistImages, catsImages, crabsImages, facesImages), 0)
        labelsList = [0] * catsImages.size(0) + [1] * crabsImages.size(0) + [2] * facesImages.size(0) + [3] * mnistImages.size(0)
        labelsList = torch.Tensor(labelsList).int()
               
        indexes = set()
        while len(indexes) < samples:
            #i1 = randint(0, imagesList.size(0) - 1)
            #i2 = randint(0, imagesList.size(0) - 1)
            i1 = randIndex([catsImages, crabsImages, facesImages, mnistImages])
            i2 = randIndex([catsImages, crabsImages, facesImages, mnistImages])
            indexes.add((i1, i2))
        indexes = [j for i in list(indexes) for j in i]

        images = torch.index_select(imagesList, 0, torch.LongTensor(indexes))
        labels = torch.index_select(labelsList, 0, torch.LongTensor(indexes))
        images = images.view(-1, 2, images.size(1), images.size(2))
        labels = labels.view(-1, 2)
        distances = sqeuclidean_wasserstein_distance(images[:, 0], images[:, 1])

        images = images.float()
        distances = distances.float()

        save_torch_big_data(images, 'images.pt', samples)
        save_torch_big_data(labels, 'labels.pt', samples)
        save_torch_big_data(distances, 'distances.pt', samples)
        
    log('%d samples count' % images.size(0))
    log('distances sum: %.2f' % distances.sum())
    log('distances min: %.2f' % distances.min())
    log('distances max: %.2f' % distances.max())

    n_train = int(images.size(0) * 0.7)
    n_valid = int(images.size(0) * 0.9)
    train_images = images[:n_train]
    train_labels = labels[:n_train]
    train_distances = distances[:n_train]

    valid_images = images[n_train:n_valid]
    valid_labels = labels[n_train:n_valid]
    valid_distances = distances[n_train:n_valid]

    test_images = images[n_valid:]
    test_labels = labels[n_valid:]
    test_distances = distances[n_valid:]

    log('finish loading')
    return (train_images, train_labels, train_distances, valid_images, valid_labels, valid_distances, test_images, test_labels, test_distances)

In [ ]:
def printMatrixDistancesLabels(distances, labels, classesCount):
    samples = distances.size(0)
    distances_matrix_sum = np.zeros((classesCount, classesCount))
    distances_matrix_count = np.zeros((classesCount, classesCount))

    for i in range(samples):
        l1 = max(labels[i])
        l2 = min(labels[i])
        distances_matrix_sum[l1][l2] += 2
        distances_matrix_sum[l1][l2] += distances[i]
        distances_matrix_count[l1][l2] += 1

    np.set_printoptions(precision=2)
    log('\n' + str(distances_matrix_sum))
    log('\n' + str(distances_matrix_count))
    log('\n' + str(distances_matrix_sum / distances_matrix_count))

#     from common.plotting import plot_mat
#     for i in range(10):
#         d = sqeuclidean_wasserstein_distance(images[i][0].unsqueeze(0), images[i][1].unsqueeze(0))[0]
#         lol = to_np(images[i])
#         lol = np.array([lol[0], lol[1]])
#         lol = np.expand_dims(lol, axis=1)
#         plot_mat(lol, cmap='gray')
#         plt.title("Distance: %.2f" % d)
#         show()

In [ ]:
def compute_error_rate(model, allX, allY, batch_size):
    i = 0
    mse = 0.0
    while i < allX.size(0):
        x = Variable(allX[i:i+batch_size])
        y = Variable(allY[i:i+batch_size])
        if CUDA:
            x = x.cuda()
            y = y.cuda()
        (outputs, _) = model(x)
        diff = (outputs - y).data
        mse = mse + torch.sum(diff ** 2)
        i = i + batch_size
    return mse / allX.size(0)

def compute_prediction(model, allX, batch_size):
    i = 0
    y = torch.FloatTensor()
    if CUDA:
        y = y.cuda()
    while i < allX.size(0):
        x = Variable(allX[i:i+batch_size])
        if CUDA:
            x = x.cuda()
        y = torch.cat((y, model(x)[0].data), 0)
        i += batch_size
    return y

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Sequential(
                nn.Conv2d(1, 20, kernel_size=3, padding=0),
                nn.ReLU()),
            nn.Sequential(
                nn.Conv2d(20, 10, kernel_size=3, padding=0),
                nn.ReLU()),
            nn.Sequential(
                nn.Conv2d(10, 5, kernel_size=5, padding=0),
                nn.ReLU()),
            Reshape(-1),
            nn.Sequential(
                nn.Linear(2000, 100),
                nn.ReLU()),
            nn.Sequential(
                nn.Linear(100, 50),
                nn.ReLU())
        )
        
        self.decoder = nn.Sequential(
            nn.Sequential(
                nn.Linear(50, 100),
                nn.ReLU()),
            nn.Sequential(
                nn.Linear(100, 5 * 32 * 32),
                nn.ReLU()),
            Reshape(5, 32, 32),
            nn.Sequential(
                nn.Conv2d(5, 10, kernel_size=1, padding=0),
                nn.ReLU()),
            nn.Sequential(
                nn.Conv2d(10, 20, kernel_size=3, padding=0),
                nn.ReLU()),
            nn.Sequential(
                nn.Conv2d(20, 1, kernel_size=3, padding=0),
                nn.ReLU()),
            nn.Softmax2d()
        )
        
    def forward(self, data):
        data1 = data[:, 0].unsqueeze(1)
        encoder1 = self.encoder(data1)
        decoder1 = self.decoder(encoder1)
        
        data2 = data[:, 1].unsqueeze(1)
        encoder2 = self.encoder(data2)
        decoder2 = self.decoder(encoder2)
            
        encoder_difference = encoder1 - encoder2
        encoder_factor = torch.torch.matmul(encoder_difference, encoder_difference.transpose(0, 1)).diag()
        
        return (encoder_factor, torch.cat((decoder1, decoder2)))
      
def loadModel(model):
    try:
        log('load model')
        model.load_state_dict(load_torch_config('cnn.pkl'))
        log('load model finished')
    except:
        log('load model error')

def trainModel(model, train_images, train_distances, valid_images, valid_distances, learning_rate, samples, batch_size, restore):
    log('start training')

    epoch = 0
    num_epochs = 1
    patience_expansion = 1.5
    best_value_error = 1000000.0
    best_params = None

    mse_criterion = nn.MSELoss()
    kl_criterion = nn.KLDivLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    if restore:
        try:
            log('restore training data')
            model.load_state_dict(load_torch_config('cnn.pkl'))
            optimizer.load_state_dict(load_torch_config('optimizer.pkl'))
            e = load_torch_config('epoch.pkl')
            epoch = e[0]
            num_epochs = e[1]
            log('restore training data finished')
        except:
            log('restore training data error')
    
    try:
        while epoch < num_epochs:
            epoch += 1
            i = 0
            while i < train_images.size(0):
                optimizer.zero_grad()
                x = Variable(train_images[i:i+batch_size])
                y = Variable(train_distances[i:i+batch_size])
                if CUDA:
                    x = x.cuda()
                    y = y.cuda()
                (distances_outputs, images_outputs) = model(x)
                loss1 = mse_criterion(distances_outputs, y)
                loss2 = kl_criterion(images_outputs, x)
                loss = loss1 + loss2
                loss.backward()
                optimizer.step()
                i = i + batch_size

            value_error = compute_error_rate(model, valid_images, valid_distances, batch_size)
            if (value_error < best_value_error):
                best_value_error = value_error
                num_epochs = int(np.maximum(num_epochs, epoch * patience_expansion + 1))
                best_params = [p.clone().cpu() for p in model.parameters()]

            log('epoch [%d/%d]' % (epoch, num_epochs))
            log('validation set errors: %.2f' % value_error)
            log('')

            save_torch_config(model.state_dict(), 'cnn.pkl')
            save_torch_config(optimizer.state_dict(), 'optimizer.pkl')
            save_torch_config(torch.Tensor([epoch, num_epochs]), 'epoch.pkl')

            with open('%s/%s' % (DST_CONFIG_DIR, CONFIG_FILE), 'w') as file:
                file.write("samples            %d\n" % (samples))
                file.write("num_epochs         %d\n" % (num_epochs))
                file.write("epoch              %d\n" % (epoch))
                file.write("batch_size         %d\n" % (batch_size))
                file.write("patience_expansion %.4f\n" % (patience_expansion))
                file.write("best_value_error   %.4f\n" % (best_value_error))
                file.write("learning_rate      %.8f\n" % (learning_rate))
                file.flush()

    except KeyboardInterrupt:
        pass

    if best_params is not None:
        model.parameters = best_params
        
    save_torch_config(model.state_dict(), 'cnn.pkl')
    save_torch_config(optimizer.state_dict(), 'optimizer.pkl')    

    log('finish training')
    
def plotScatter(title, model, x, y, batch_size):
    exact = np.arange(0.0, 45.0, step=0.1)
    error = compute_error_rate(model, x, y, batch_size)
    log('MSE = %.2f' % error)
    _, sorted_indices = torch.sort(y)
    y1 = to_np(torch.index_select(y, 0, sorted_indices))
    y2 = to_np(torch.index_select(compute_prediction(model, x, batch_size).cpu(), 0, sorted_indices))

    mean1 = np.array([])
    mean2 = np.array([])
    p10_1 = np.array([])
    p10_2 = np.array([])
    p90_1 = np.array([])
    p90_2 = np.array([])
    
    for x in np.arange(1.25, 46.25, 2.5):
        ind = (x - 1.25 < y1) & (y1 < x + 1.25)
        diffY = y1[ind] - y2[ind]
        if diffY.shape[0] == 0:
            continue
        error_value = (diffY ** 2).mean()
        mean1 = np.append(mean1, x)
        mean2 = np.append(mean2, x - error_value)
        
        p10_1 = np.append(p10_1, x)
        p10_2 = np.append(p10_2, x - np.percentile(diffY, 10))
        
        p90_1 = np.append(p90_1, x)
        p90_2 = np.append(p90_2, x - np.percentile(diffY, 90))
        
    fig = plt.figure(figsize = (10, 8))
    plt.plot(exact, exact, '-', label = 'Exact prediction')
    plt.plot(mean1, mean2, '-x', label = 'Mean prediction')
    plt.plot(p10_1, p10_2, '-x', label = '10th percentile')
    plt.plot(p90_1, p90_2, '-x', label = '90th percentile')
    plt.ylabel('Predicted Wass. distance')
    plt.xlabel('True Wass. distance')
    plt.title(title + 'MSE=%.2f' % error)
    plt.legend()
    plt.grid(True)

In [ ]:
CUDA = True
DST_CONFIG_DIR = 'mnist'
DST_BIG_DATA_DIR = '/pio/scratch/1/i233123/data_mnist'
LOG_FILE = 'stdout.txt'
CONFIG_FILE = 'config.txt'

# if os.path.exists(LOG_FILE):
#     os.remove(LOG_FILE)

# if not os.path.exists('/pio/scratch/1/i233123/'):
#     os.mkdir('/pio/scratch/1/i233123/')
        
def run_mnist(samples, learning_rate, batch_size, restore, train):
    global DST_CONFIG_DIR
    DST_CONFIG_DIR = ('%s/%.6f' % (DST_CONFIG_DIR, learning_rate)).replace(".", "#")
    (tr_i, tr_l, tr_d, v_i, v_l, v_d, ts_i, ts_l, ts_d) = load_mnist(samples)
    printMatrixDistancesLabels(tr_d, tr_l, 10)
    
    model = CNN()
    if CUDA:
        model.cuda()

    if restore:
        loadModel(model)
    if train:
        trainModel(model, tr_i, tr_d, v_i, v_d, learning_rate, samples, batch_size, restore)
    model.eval()
    
    plotScatter('MNIST test set ', model, ts_i, ts_d, batch_size)
    
run_mnist(1000000, 0.0005, 1000, True, False)

In [ ]:
CUDA = True
DST_CONFIG_DIR = 'doodle'
DST_BIG_DATA_DIR = '/pio/scratch/1/i233123/data_doodle'
LOG_FILE = 'stdout.txt'
CONFIG_FILE = 'config.txt'

# if os.path.exists(LOG_FILE):
#     os.remove(LOG_FILE)

# if not os.path.exists('/pio/scratch/1/i233123/'):
#     os.mkdir('/pio/scratch/1/i233123/')
        
def run_doodle(samples, learning_rate, batch_size, restore, train):
    global DST_CONFIG_DIR
    DST_CONFIG_DIR = ('%s/%.6f' % (DST_CONFIG_DIR, learning_rate)).replace(".", "#")
    (tr_i, tr_l, tr_d, v_i, v_l, v_d, ts_i, ts_l, ts_d) = load_doodle(samples)
    printMatrixDistancesLabels(tr_d, tr_l, 4)
        
    model = CNN()
    if CUDA:
        model.cuda()

    if restore:
        loadModel(model)
    if train:
        trainModel(model, tr_i, tr_d, v_i, v_d, learning_rate, samples, batch_size, restore)
    model.eval()
    
    plotScatter('Doodle test set ', model, ts_i, ts_d, batch_size)
    
run_doodle(1000000, 0.0005, 1000, False, True)